# 2.06 - Defining an Objective Function
This workbook takes us through the formulation of an objective function to help indicate the 'goodness' of a night, in terms of glycaemic control, where the night is defined as the period between 22:00 and 06:00 the following day. To recap, the following measures have been studied and will contribute as components to the objective function. These are recorded at each 30 minutes interval in our dataset:
- Blood Glucose Mean
- Blood Glucose Standard Deviation
- Coefficient of Variation
- Intervals Outside Level 1 Target Range
- Intervals Outside Level 2 Target Range
- Amplitude of Glycaemic Variability (absolute value above 1 SD from the mean)
- Peaks of Carbohydrate Intake

The goal is still to produce a continuous variable where the score indicates the quality of the night, encompassing the physiological impact observed through glycaemic metrics and carbohydrate intake, with a higher score indicating greater disturbance.

## Method - Overview of Objective Function Components

For each night (22:00 to 06:00), the 30-minutes intervals are aggregated data for the following metrics:
- Blood Glucose Mean (BG_Mean_Night): The average of all 30-minute BG means throughout the night. A higher average BG suggests potential discomfort or dysregulation.
- Blood Glucose Standard Deviation (BG_SD_Night): The average of all 30-minute BG standard deviations (or the overall standard deviation of all BG readings during the night). High variability can indicate instability and potential physiological stress.
- NOTE: The Law of Total Variance for Aggregated Data (for which we have at the 30 minute intervals, from the original irregular time series provides a more statistically robust measure of variability than the simple average of standard deviations):
  - $\text{Var}(X) = E[\text{Var}(X|Y)] + \text{Var}(E[X|Y])$
  - This means that the variance of the aggregated data is the sum of the expected variance within each group and the variance of the group means. The theoretical formula for the overall standard deviation of the aggregated data is as follows:
Let $X$ be the original, high-frequency blood glucose readings.
Let $Y$ represent the 30-minute time intervals (or groups).
Let $g_i$ be the blood glucose mean for interval $i$.
Let $s_i$ be the blood glucose for interval $i$.
Let $s_i^2$ be the variance within interval $i$ (which is $s_i$ squared).
Let $n_i$ be the number of original readings that were aggregated into interval $i$.
Leading to the formula for variance:

$\text{Var}_{overall} = \frac{\sum_{i=1}^K n_i \cdot s_i^2}{\sum_{i=1}^K} + \frac{\sum_{i=1}^K n_i \cdot (g_i-\bar G)^2}{\sum_{i=1}^K n_i}$

And standard deviation:
$\sigma_{overall} = \sqrt{\text{Var}_{overall}}$

Where:
- $K$ is the total number of 30-minute intervals in the night.
- $\bar G$ is the overall mean of all blood glucose readings during the night.

- Coefficient of Variation (CV_Night): The average of the 30-minute Coefficient of Variation values. This normalizes variability to the mean, providing a robust measure of relative glucose fluctuations, which can be disturbing.
- Intervals Outside Level 1 Target Range (L1_Excursions_Night): The sum or count of 30-minute intervals where a Level 1 (clinically defined) excursion occurred. These indicate deviations from a generally acceptable range, potentially causing discomfort or impacting restorative processes.
- Intervals Outside Level 2 Target Range (L2_Excursions_Night): The sum or count of 30-minute intervals where a Level 2 (more severe) excursion occurred. These represent more significant deviations, very likely disruptive.
- Mean Amplitude of Glycaemic Variability (MAGE): The sum or average of the absolute amplitude values (above 1 SD from the mean). These capture significant, rapid swings in glucose, which are often associated with physiological stress or symptoms that could disturb sleep, even if within clinical thresholds. If the dataset is very sparse for this, consider summing non-zero amplitudes or taking the maximum amplitude for the night to highlight the presence of any large swings.
- Peaks of Carbohydrate Intake (COB_Peaks): The count of peaks of COB for the night.

## Objective Function Definition
Let the following notation be used:
- $G_t$:Blood glucose reading at time t.
- $T$: Total number of time intervals in the observed period.
- $N$: The set of all glucose readings during the nocturnal period, $\{G_1, G_2, ..., G_T\}$.
- $G_{L1\_lower}$: Lower threshold for Level 1 excursion.
- $G_{L1\_upper}$: Upper threshold for Level 1 excursion.
- $G_{L2\_lower}$: Lower threshold for Level 2 excursion.
- $G_{L2\_upper}$: Upper threshold for Level 2 excursion.

All features are scaled using `Standard Scaler` to ensure they contribute equally to the objective function. The scaling is done using the mean and standard deviation of each feature across all nights:

$X_{scaled} = \frac{X - \mu_X}{\sigma_X}$

Where $X$ is the original feature value, $\mu_X$ is the mean of the feature across all nights, and $\sigma_X$ is the standard deviation of the feature across all nights.

1. Minimising Variance Using Standard Deviation

$\mu_G = \frac{1}{T} \sum_{t=1}^{T} G_t$ is the mean of the glucose readings during the nocturnal period, thus the standard deviation is calculated as $\sigma_G = \sqrt{\frac{1}{T} \sum_{t=1}^{T} (G_t - \mu_G)^2}$.

3. Minimising Glycaemic Variability Using Coefficient of Variation

Standard deviation is a measure of variability, but it does not account for the mean level of glucose. The Coefficient of Variation (CV) is defined as $CV = \frac{\sigma_G}{\mu_G}$, which normalises the standard deviation by the mean glucose level, providing a relative measure of variability.

4. Minimising Excursions Outside Target Ranges

The ranges used here provide a clinically relevant context for evaluating glucose levels. The Level 1 excursion is defined as $G_{L1\_lower} \leq G_t \leq G_{L1\_upper}$, and the Level 2 excursion is defined as $G_{L2\_lower} \leq G_t \leq G_{L2\_upper}$. The number of intervals outside these ranges can be counted as follows:
- $L1\_Excursions = \sum_{t=1}^{T} \mathbb{1}_{G_t < G_{L1\_lower} \lor G_t > G_{L1\_upper}}$
- $L2\_Excursions = \sum_{t=1}^{T} \mathbb{1}_{G_t < G_{L2\_lower} \lor G_t > G_{L2\_upper}}$

5. Minimising Amplitude of Glycaemic Variability

....

6. Minimising Peaks of Carbohydrate Intake

Carbohydrate intake is a significant factor in glucose regulation. Peaks of carbohydrate intake can be counted as the number of times the carbohydrate intake exceeds a certain threshold during the nocturnal period. This can be defined as:
- $COB\_Peaks = \sum_{t=1}^{T} \mathbb{1}_{COB_t > COB\_threshold}$, where $COB_t$ is the carbohydrate intake at time t.
No carbohydrate height parameter is set as any peak is considered an indicator that the person is awake and eating, which is not ideal for a good night.

The objective function $J$ can be defined as a weighted sum of these components, $f$ be a features used in the objective function from the set $\{f_1, f_2, \dots , f_k\}$ and $w_k$ be a weight applied to the feature:

$J = \frac{\sum_{k=1}^K w_k \cdot f_k}{K}$


In [12]:
%load_ext autoreload
%autoreload 2

from src.nights import Nights, consolidate_df_from_nights
from datetime import time
import pandas as pd
from src.config import PROCESSED_DATA_DIR
from src.features import FeatureSet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df_all = pd.read_parquet(PROCESSED_DATA_DIR / 'night_clusters.parquet')
df_all['time'] = df_all.index.get_level_values('datetime').time

sample_rate = 30
nights_objects = []

night_start = time(22, 0)
morning_end = time(6, 0)

for zip_id, df_ind in df_all.groupby('id'):
    df_ind_reset = df_ind.reset_index(level='id', drop=True)
    nights_objects.append(
        Nights(df=df_ind_reset, zip_id=zip_id, night_start=night_start,
               morning_end=morning_end, sample_rate=sample_rate))

df_overnight = consolidate_df_from_nights(nights_objects)

In [14]:
features = FeatureSet(df_overnight, sample_rate=sample_rate)
df_features = features.get_all_features(scale=False)

In [15]:
features_scaled = FeatureSet(df_overnight, sample_rate=sample_rate)
df_scaled = features_scaled.get_all_features(scale=True)

Scaling ['iob mean', 'cob mean', 'bg mean', 'bg min', 'iob max', 'cob max', 'bg max', 'bg std', 'bg count', 'day_type_weekday', 'day_type_weekend', 'iob mean_rate_of_change', 'cob mean_rate_of_change', 'bg mean_rate_of_change', 'iob mean hourly_mean', 'cob mean hourly_mean', 'bg mean hourly_mean', 'bg max_peaks_above_mean', 'iob max_peaks_above_mean', 'cob max_peaks_above_mean', 'hour_of_day', 'hour_sin', 'hour_cos', 'l1_hypo', 'l1_hyper', 'l2_hypo', 'l2_hyper', 'excursion_amplitude', 'excursion_flag', 'cob_peaks'] columns


In [16]:
feature_cols = ['bg mean', 'l1_hypo', 'l1_hyper', 'l2_hypo', 'l2_hyper', 'excursion_amplitude', 'excursion_flag', 'cob_peaks']

In [17]:
df_scaled.columns

Index(['iob mean', 'cob mean', 'bg mean', 'bg min', 'iob max', 'cob max',
       'bg max', 'bg std', 'bg count', 'day_type_weekday', 'day_type_weekend',
       'iob mean_rate_of_change', 'cob mean_rate_of_change',
       'bg mean_rate_of_change', 'iob mean hourly_mean',
       'cob mean hourly_mean', 'bg mean hourly_mean',
       'bg max_peaks_above_mean', 'iob max_peaks_above_mean',
       'cob max_peaks_above_mean', 'hour_of_day', 'hour_sin', 'hour_cos',
       'l1_hypo', 'l1_hyper', 'l2_hypo', 'l2_hyper', 'excursion_amplitude',
       'excursion_flag', 'cob_peaks', 'night_start_date', 'cluster'],
      dtype='object')

Now we have the set of features at a time interval level, we need to aggregate them at a night level, which is

In [22]:
from src.objective_function import ObjectiveFunction
obj_func = ObjectiveFunction(df_overnight, sample_rate=sample_rate)

In [23]:
features = obj_func.aggregate_features()

In [24]:
scaled = obj_func.scale_features()

In [25]:
scaled

array([[ 0.21609028,  0.11886728,  0.23464814, ..., -0.36806778,
        -0.00430508, -0.52547966],
       [-0.59204031, -0.44178981, -0.52318083, ..., -0.36806778,
        -1.94420737, -0.52547966],
       [ 0.16815308, -0.01530752,  0.25980251, ..., -0.36806778,
        -0.05726763, -0.52547966],
       ...,
       [-0.22878275,  1.19611346, -0.72003578, ..., -0.36806778,
        -0.47613622, -0.52547966],
       [ 1.1211605 ,  2.01343765,  0.19340405, ...,  1.80382655,
         2.31159563,  2.86257802],
       [-0.07100272,  2.31402789, -0.86468816, ...,  0.17490581,
         0.15540244, -0.52547966]], shape=(922, 7))

In [26]:
obj_func.get_objective_function_scores()

0
id       night_start_date          
15558575 2018-04-17       -0.094272
         2018-04-21       -0.750455
         2018-04-22       -0.199512
         2018-04-27        0.351134
         2018-04-30        0.287384
...                             ...
97417885 2018-02-09        1.539295
         2018-02-10        0.141781
         2018-02-11       -0.170095
         2018-02-16        1.688289
         2018-02-17        0.610782

[922 rows x 1 columns]